In [1]:
import os
from lxml import etree
from collections import OrderedDict

In [2]:
# path to video mp4
video_path = "replace_me"

# path to CVAT export xml
annotation_path = "replace_me"

# path to tracks directory
tracks_location = "replace_me"

In [3]:
root = etree.parse(annotation_path).getroot()
annotated = dict()
track2end = {}

for track in root.iterfind("track"):
    track_id = int(track.attrib["id"])

    for box in track.iter("box"):
        frame_id = int(box.attrib["frame"])
        keyframe = int(box.attrib["keyframe"])

        if keyframe == 1:
            track2end[track_id] = frame_id

for track in root.iterfind("track"):
    track_id = int(track.attrib["id"])

    for box in track.iter("box"):
        frame_id = int(box.attrib["frame"])
        keyframe = int(box.attrib["keyframe"])

        if frame_id <= track2end[track_id]:
            if annotated.get(track_id) is None:
                annotated[track_id] = OrderedDict()
                
            scaling_factor = 3

            annotated[track_id][frame_id] = [int(float(box.attrib["xtl"])*scaling_factor),
                                                int(float(box.attrib["ytl"])*scaling_factor),
                                                int(float(box.attrib["xbr"])*scaling_factor),
                                                int(float(box.attrib["ybr"])*scaling_factor), keyframe]

In [4]:
xml_page = etree.Element("annotations")
etree.SubElement(xml_page, "version").text = "1.1"

for track_id in annotated.keys():
    xml_track = etree.Element("track", id=str(track_id), label="Grevy", source="manual")

    for frame_id in annotated[track_id].keys():
        if frame_id == sorted(annotated[track_id].keys())[-1]:
            outside = "1"
        else:
            outside = "0"

        xml_box = etree.Element("box", frame=str(frame_id), outside=outside, occluded="0",
                                keyframe=str(annotated[track_id][frame_id][4]),
                                xtl=f"{annotated[track_id][frame_id][0]:.2f}",
                                ytl=f"{annotated[track_id][frame_id][1]:.2f}",
                                xbr=f"{annotated[track_id][frame_id][2]:.2f}",
                                ybr=f"{annotated[track_id][frame_id][3]:.2f}", z_order="0")

        xml_track.append(xml_box)

    if len(annotated[track_id].keys()) > 0:
        xml_page.append(xml_track)


# Parse the original XML file
original_tree = etree.parse(annotation_path)
original_root = original_tree.getroot()

# Extract the 'meta' element
meta = original_root.find('meta')

# Append 'meta' to the new XML document
xml_page.append(meta)

xml_document = etree.ElementTree(xml_page)

xml_document.write(f"{tracks_location}/tracks_.xml", xml_declaration=True, pretty_print=True, encoding="utf-8")

# Extract tracks with KABR tools

In [ ]:
os.system(f"tracks_extractor --video {video_path} --annotation {tracks_location}/tracks_.xml")